In [1]:
# !/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import pickle
#import plotly.express as px
#import plotly.figure_factory as ff
#import shap
import datetime
import logging
import yaml
from sklearn.impute import KNNImputer
from sklearn.metrics import recall_score, precision_score #, accuracy_score
# from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix
# from sklearn.metrics import jaccard_score, roc_auc_score, average_precision_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold#, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from category_encoders.count import CountEncoder

In [3]:
from training_functions import *

In [6]:
df_train = get_data(name_sav='wA_train.sav', path='../outputs/')

Se hará una pequeña competencia entre un modelo clásico de clasificación, es decir, `RandomForest` y un modelo más comnplejo como `HistogramGradientBoosting`.